In [1]:
import glob
from glob import glob
import shutil
import requests
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import traceback
import logging
import logging.handlers
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy as sa
from sqlalchemy import create_engine, event
import json
import time as ti
import datetime
from datetime import datetime, date, timedelta
import re
import os
import asyncio
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import ctypes
import threading

global processlist, df
conn = False
con = None

filepath = r'DataAndDuplicateReportAutomation'
try:
    
    logging.basicConfig(filename = 'RRMDuplicateExceptionsFinalSection.log',
                        filemode='a',
                        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                        datefmt='%H:%M:%S',
                        level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')
    


    def check_files():
        global processlist
        processlist = []
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith("DataAndDuplicateReport"):
                print(f'\n{file}')
                processlist += [file]
            elif file.startswith(("Archive","Analysis","Temp")):
                pass
            elif ~file.startswith("DataAndDuplicateReport"):
                print('incorrect file name is present.')
        if processlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processlist += ['None']
        else:
            pass
        return processlist
    
    def convert_to_CSV(doc):
        #global df
        pd.read_excel(f'{filepath}'+'/'+f'{doc}', sheet_name='Daily data report').to_csv(f'{filepath}'+'/'+'DataAndDuplicateReport' + '_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.csv', encoding = 'utf-8', index = False)
        shutil.move(f'{filepath}'+'/'+f'{doc}', 'DataAndDuplicateReportAutomation/Archive/xlsx/'+f'{doc}')
        #for f in glob('DataAndDuplicateReportAutomation/DataAndDuplicateReport*.csv'):
            #df = pd.read_csv(f)
        #display(df)
        return #df
    
    def process_CSV():
        global processlist, df
        processlist = []
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith("DataAndDuplicateReport") and file.endswith(".csv"):
                print(f'\n{file}')
                processlist += [file]
            elif file.startswith(("Archive","Analysis","Temp")):
                pass
            elif ~file.startswith("DataAndDuplicateReport"):
                print('incorrect file name is present.')
        if processlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processlist += ['None']
        else:
            pass
        return processlist

    def duplication_extraction():
        
        global df1pre, df1, df2
        
        pd.set_option('display.max_columns', None)
    
        df1pre = pd.read_csv(f'DataAndDuplicateReportAutomation/{processlist[0]}')
        
        df1 = df1pre.sort_values(by=['PersonPartyID','CreatedDatetime'], ascending = True)
        
        today = date.today()
        first = today.replace(day=1)
        last_month = first - timedelta(days=1)
        tenth_of_current_month = today.replace(day=10)
        first_of_previous_month = last_month.replace(day=1)
        next_month = today.replace(day=28) + timedelta(days=4)
        next_month = next_month - timedelta(days=next_month.day)
        
        df1['CreatedDatetime']= pd.to_datetime(df1['CreatedDatetime']).dt.date
        df1['EffectiveDate'] = pd.to_datetime(df1['EffectiveDate']).dt.date
        df1.loc[(df1['ExpiryDate'].isna()), 'ExpiryDate'] = '1/1/1990'
        df1['ExpiryDate'] = pd.to_datetime(df1['ExpiryDate']).dt.date
        df1['LastModifiedDatetime'] = pd.to_datetime(df1['LastModifiedDatetime']).dt.date
        
        df1 = df1.where(pd.notnull(df1), None)
        
        df1.loc[~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['CreatedDatetime'].transform(max) == df1['CreatedDatetime']), 'CreatedDatetime_status'] = 'Previous'
        df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['CreatedDatetime'].transform(max) == df1['CreatedDatetime']), 'CreatedDatetime_status'] = 'Latest'
            
        df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','RateBasis'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & ~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(min) == df1['ExpiryDate']), 'ExpiryDate_status'] = 'Latest'
        df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & ((df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','RateBasis'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(min) == df1['ExpiryDate'])), 'ExpiryDate_status'] = 'Current'
        df1.loc[~(df1['CreatedDatetime_status'] == 'Latest') & ((df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis',])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(min) == df1['ExpiryDate'])), 'ExpiryDate_status'] = 'Previous CD but Max & Min Exp Date Error'
        df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis',])['ExpiryDate'].transform(max) != df1['ExpiryDate']), 'ExpiryDate_status'] = 'Latest CD but not Max Exp Date Error'
        df1.loc[~(df1['CreatedDatetime_status'] == 'Latest') & ((df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis',])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & ~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(min) == df1['ExpiryDate'])), 'ExpiryDate_status'] = 'Previous CD but Max & not Min Exp Date Error'
        df1.loc[~(df1['CreatedDatetime_status'] == 'Latest') & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis',])['ExpiryDate'].transform(max) != df1['ExpiryDate']), 'ExpiryDate_status'] = 'Previous CD and not Max Exp Date Error'
        df1.loc[pd.to_datetime(df1['ExpiryDate']) == '1990-01-01', 'ExpiryDate_status'] += ' & Null Placeholder'
        df1.loc[(pd.to_datetime(df1['ExpiryDate']) != '1990-01-01') & (pd.to_datetime(date.today()) >= df1['ExpiryDate']), 'ExpiryDate_status'] += ' & Expired'
        
        df1.loc[~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max) == df1['EffectiveDate']), 'EffectiveDate_status'] = 'Previous EffectiveDate'
        df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max) == df1['EffectiveDate']), 'EffectiveDate_status'] = 'Current EffectiveDate'
        df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max) == df1['EffectiveDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['LastModifiedDatetime'].transform(max) == df1['LastModifiedDatetime']), 'EffectiveDate_status'] += ' & Latest MD'
        df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max) == df1['EffectiveDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['LastModifiedDatetime'].transform(max) != df1['LastModifiedDatetime']), 'EffectiveDate_status'] += ' & Previous MD'
        
        df1.loc[(df1['EffectiveDate']) < (df1['CreatedDatetime']), 'EffectiveDate_status'] += ' & Eff Date less than CD Anomaly'
        df1.loc[(((df1['EffectiveDate']) >= (df1['ExpiryDate'])) & ~(pd.to_datetime(df1['ExpiryDate']) == '1990-01-01')), 'EffectiveDate_status'] += ' & Eff Date greater than/Equal to Exp Date Error'
        
        todayv2 = datetime.strptime(str(today), '%Y-%m-%d').strftime('%Y-%m-%d')
        todayv3 = datetime.strptime(str(today), '%Y-%m-%d').strftime('%m-%d')
        last_monthv2 = datetime.strptime(str(last_month), '%Y-%m-%d').strftime('%Y-%m-%d')
        df1['ExpiryDateFirstOfPreviousMonthStr'] = ((df1['ExpiryDate']) - relativedelta(months=1)).astype('str')
        df1['ExpiryDateMaxEffDateMinus1'] = df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max)
        df1['ExpiryDateMaxEffDateMinus1'] = ((df1['ExpiryDateMaxEffDateMinus1']) - relativedelta(days=1)).astype('str')
        df1['Update_Statement'] = 'update resourcepersonrate set'
        df1.loc[df1['ExpiryDate_status'].str.contains('Expired') , 'Update_Statement'] += " expirydate = '1990-01-01'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD & Eff Date greater than/Equal to Exp Date Error'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '"+df1['ExpiryDateMaxEffDateMinus1']+"'"
        df1.loc[df1['Update_Statement'] == 'update resourcepersonrate set' , 'Update_Statement'] = ''
        df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f", lastmodifieddatetime = '{todayv2}'"
        df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f", comments = comments +' | edit from {todayv3}'"
        df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f" where resourcepersonratesk = '"+df1['Resourcepersonratesk'].astype('str')+"'"
        
        df2 = df1[['PersonPartyID','LegalEntityPartyID','rateType','EffectiveDate','ExpiryDate','CreatedDatetime','LastModifiedDatetime','RateBasis','CreatedDatetime_status','ExpiryDate_status','EffectiveDate_status','Update_Statement']]
        
        #,'PartyID_RateRule','RateRule_Eff_Date','RateRule_Exp_Date','Rate_Basis_Count','ExpiryDate_update_S1','ExpiryDate_update_S2','ExpiryDate_update_equaldates',
        
        #UpdExp = "update resourcepersonrate set expirydate = '2022-09-30', lastmodifieddatetime = No, comments = comments +':11/8' where resourcepersonratesk = 20550402"
        #UpdLMD = "update resourcepersonrate set lastmodifieddatetime = getdate(), comments = '11/8 LMD' where resourcepersonratesk = 27151709"
        
        display(df2)
        
        return
                    
    check_files()
    if 'None' not in processlist:
        print('\ndocuments available for processing\n')
        print(processlist)
        for doc in enumerate(processlist):
            try:
                gstart_time = datetime.now()
                convert_to_CSV(doc[1])
                gend_time = datetime.now()
                process_CSV()
                #duplication_extraction()
                print("")
                print('\nDuration: {}'.format(gend_time - gstart_time))
                logging.info('\nDuration: {}'.format(gend_time - gstart_time))
            except Exception as e:
                print("")
                print(e)
                logging.exception('\n')
                logging.exception("message")
            for i in range(10, -1, -1):
                    print(f"{i} seconds until next document is processed", end = "\r")
                    ti.sleep(1)
    else:
        print('documents not available for processing')
        pass

#exceptions 
except (Exception, pyodbc.DatabaseError) as error:
        print(error)
        logging.exception("message")
        pass
    
except requests.exceptions.HTTPError as errh:
    print("Http Error:",  errh)
    conn = False
    logging.exception("message")
    
except requests.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
    conn = False
    logging.exception("message")
    
except requests.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
    conn = False
    logging.exception("message")
    
except requests.exceptions.RequestException as erru:
    print("Unidentified Request Exception:", erru)
    conn = False
    logging.exception("message")
        
finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')
    if conn == True:
        close_connection()
        cur.close()

Analysis
Archive
DataAndDuplicateReport_2022-11-21_19-01-29.csv

DataAndDuplicateReport_2022-11-21_19-01-29.csv
Temp

documents available for processing

['DataAndDuplicateReport_2022-11-21_19-01-29.csv']

Excel file format cannot be determined, you must specify an engine manually.


In [4]:
import glob
from glob import glob
import shutil
import requests
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import traceback
import logging
import logging.handlers
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy as sa
from sqlalchemy import create_engine, event
import json
import time as ti
import datetime
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import re
import os
import asyncio
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import ctypes
import threading

def duplication_extraction():
    
    global df1pre, df1, df2_filter
    
    pd.set_option('display.max_columns', None)

    df1pre = pd.read_csv(f'DataAndDuplicateReportAutomation/{processlist[0]}')
    
    df1 = df1pre.sort_values(by=['PersonPartyID','CreatedDatetime'], ascending = True)
    
    today = date.today()
    first = today.replace(day=1)
    last_month = first - timedelta(days=1)
    tenth_of_current_month = today.replace(day=10)
    first_of_previous_month = last_month.replace(day=1)
    next_month = today.replace(day=28) + timedelta(days=4)
    next_month = next_month - timedelta(days=next_month.day)
    
    df1['CreatedDatetime']= pd.to_datetime(df1['CreatedDatetime']).dt.date
    df1['EffectiveDate'] = pd.to_datetime(df1['EffectiveDate']).dt.date
    df1.loc[(df1['ExpiryDate'].isna()), 'ExpiryDate'] = '1/1/1990'
    df1['ExpiryDate'] = pd.to_datetime(df1['ExpiryDate']).dt.date
    df1['LastModifiedDatetime'] = pd.to_datetime(df1['LastModifiedDatetime']).dt.date
    
    df1 = df1.where(pd.notnull(df1), None)
    
    df1.loc[~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['CreatedDatetime'].transform(max) == df1['CreatedDatetime']), 'CreatedDatetime_status'] = 'Previous'
    df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['CreatedDatetime'].transform(max) == df1['CreatedDatetime']), 'CreatedDatetime_status'] = 'Latest'
        
    df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & ~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['ExpiryDate'].transform(min) == df1['ExpiryDate']), 'ExpiryDate_status'] = 'Latest'
    df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & ((df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['ExpiryDate'].transform(min) == df1['ExpiryDate'])), 'ExpiryDate_status'] = 'Current'
    df1.loc[~(df1['CreatedDatetime_status'] == 'Latest') & ((df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['ExpiryDate'].transform(min) == df1['ExpiryDate'])), 'ExpiryDate_status'] = 'Previous CD but Max & Min Exp Date Error'
    df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['ExpiryDate'].transform(max) != df1['ExpiryDate']), 'ExpiryDate_status'] = 'Latest CD but not Max Exp Date Error'
    df1.loc[~(df1['CreatedDatetime_status'] == 'Latest') & ((df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & ~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['ExpiryDate'].transform(min) == df1['ExpiryDate'])), 'ExpiryDate_status'] = 'Previous CD but Max & not Min Exp Date Error'
    df1.loc[~(df1['CreatedDatetime_status'] == 'Latest') & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['ExpiryDate'].transform(max) != df1['ExpiryDate']), 'ExpiryDate_status'] = 'Previous CD and not Max Exp Date Error'
    df1.loc[pd.to_datetime(df1['ExpiryDate']) == '1990-01-01', 'ExpiryDate_status'] += ' & Null Placeholder'
    df1.loc[(pd.to_datetime(df1['ExpiryDate']) != '1990-01-01') & (pd.to_datetime(date.today()) >= df1['ExpiryDate']), 'ExpiryDate_status'] += ' & Expired'
    
    df1.loc[~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['EffectiveDate'].transform(max) == df1['EffectiveDate']), 'EffectiveDate_status'] = 'Previous EffectiveDate'
    df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['EffectiveDate'].transform(max) == df1['EffectiveDate']), 'EffectiveDate_status'] = 'Current EffectiveDate'
    df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['EffectiveDate'].transform(max) == df1['EffectiveDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['LastModifiedDatetime'].transform(max) == df1['LastModifiedDatetime']), 'EffectiveDate_status'] += ' & Latest MD'
    df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['EffectiveDate'].transform(max) == df1['EffectiveDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['LastModifiedDatetime'].transform(max) != df1['LastModifiedDatetime']), 'EffectiveDate_status'] += ' & Previous MD'
    
    df1.loc[(df1['EffectiveDate']) < (df1['CreatedDatetime']), 'EffectiveDate_status'] += ' & Eff Date less than CD Anomaly'
    df1.loc[(((df1['EffectiveDate']) >= (df1['ExpiryDate'])) & ~(pd.to_datetime(df1['ExpiryDate']) == '1990-01-01')), 'EffectiveDate_status'] += ' & Eff Date greater than/Equal to Exp Date Error'

    todayv2 = datetime.strptime(str(today), '%Y-%m-%d').strftime('%Y-%m-%d')
    todayv3 = datetime.strptime(str(today), '%Y-%m-%d').strftime('%m-%d')
    last_monthv2 = datetime.strptime(str(last_month), '%Y-%m-%d').strftime('%Y-%m-%d')
    df1['ExpiryDateFirstOfPreviousMonthStr'] = ((df1['ExpiryDate']) - relativedelta(months=1)).astype('str')
    df1['ExpiryDateMaxEffDateMinus1'] = df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis','InternalpersonCategoryname'])['EffectiveDate'].transform(max)
    df1['ExpiryDateMaxEffDateMinus1'] = ((df1['ExpiryDateMaxEffDateMinus1']) - relativedelta(days=1)).astype('str')
    df1['Update_Statement'] = 'update resourcepersonrate set'
    df1.loc[df1['ExpiryDate_status'].str.contains('Expired') , 'Update_Statement'] += " expirydate = '1990-01-01'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD & Eff Date greater than/Equal to Exp Date Error'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '"+df1['ExpiryDateMaxEffDateMinus1']+"'"
    df1.loc[df1['Update_Statement'] == 'update resourcepersonrate set' , 'Update_Statement'] = ''
    df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f", lastmodifieddatetime = '{todayv2}'"
    df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f", comments = comments +' | edit from {todayv3}'"
    df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f" where resourcepersonratesk = '"+df1['Resourcepersonratesk'].astype('str')+"'"
    
    df2_filter = df1[['PersonPartyID','LegalEntityPartyID','rateType','EffectiveDate','ExpiryDate','CreatedDatetime','LastModifiedDatetime','RateBasis','CreatedDatetime_status','ExpiryDate_status','EffectiveDate_status','Update_Statement']]
    
    display(df2_filter)
    
    return

In [7]:
display(df2_filter)

,PersonPartyID,LegalEntityPartyID,rateType,EffectiveDate,ExpiryDate,CreatedDatetime,LastModifiedDatetime,RateBasis,CreatedDatetime_status,ExpiryDate_status,EffectiveDate_status,Update_Statement
94673,0000FB73-0624-4E47-A87B-72BC7D1DA06F,F30ABFAB-4C61-4F45-B40C-1AB13706FD3C,Cost,2022-10-01,1990-01-01,2022-10-04,2022-10-05,Resource Role,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,
94674,0000FB73-0624-4E47-A87B-72BC7D1DA06F,F30ABFAB-4C61-4F45-B40C-1AB13706FD3C,Scale,2022-10-01,1990-01-01,2022-10-04,2022-10-05,Resource Role,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,
48373,000235DB-D90F-4881-8774-8BB5C9D8EB76,5715AD3E-BF5D-46A2-A831-E68192667608,Scale,2022-09-12,1990-01-01,2022-09-01,2022-09-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
48374,000235DB-D90F-4881-8774-8BB5C9D8EB76,5715AD3E-BF5D-46A2-A831-E68192667608,Cost,2022-09-12,1990-01-01,2022-09-01,2022-09-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
104336,0005E8E3-8346-4353-8467-E99C1824F9C2,14EE24E4-336A-4C9D-8A3F-CC5FF691C6CE,Cost,2022-09-01,1990-01-01,2022-09-01,2022-09-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
...,...,...,...,...,...,...,...,...,...,...,...,...
25651,FFFCB7DF-4463-4344-9F39-530FD1267597,F21F2D2F-FF8B-4792-9271-839A91658589,Scale,2022-07-01,1990-01-01,2022-07-01,2022-07-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
87692,FFFEE8EE-33F8-4B1F-8E59-F89F156F6854,FF6D5AF8-9AE6-47D9-8317-EAA8B0E2E7B9,Scale,2022-07-01,1990-01-01,2022-07-02,2022-07-02,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,
87693,FFFEE8EE-33F8-4B1F-8E59-F89F156F6854,FF6D5AF8-9AE6-47D9-8317-EAA8B0E2E7B9,Cost,2022-08-01,1990-01-01,2022-07-19,2022-07-21,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
3132,FFFEF5FC-F5AF-4BE4-A737-896EB5C175EF,B886A7F0-9C89-48A9-BE0B-EB2337697C8D,Scale,2022-04-22,1990-01-01,2022-04-21,2022-04-21,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,


In [9]:
import glob
from glob import glob
import shutil
import requests
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import traceback
import logging
import logging.handlers
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy as sa
from sqlalchemy import create_engine, event
import json
import time as ti
import datetime
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import re
import os
import asyncio
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import ctypes
import threading

def duplication_extraction_withoutcategory():
    
    global df1pre, df1, df2, df3_nofilter
    
    pd.set_option('display.max_columns', None)

    df1pre = pd.read_csv(f'DataAndDuplicateReportAutomation/{processlist[0]}')

    today = date.today()
    first = today.replace(day=1)
    last_month = first - timedelta(days=1)
    tenth_of_current_month = today.replace(day=10)
    first_of_previous_month = last_month.replace(day=1)
    next_month = today.replace(day=28) + timedelta(days=4)
    next_month = next_month - timedelta(days=next_month.day)
    
    df1['CreatedDatetime']= pd.to_datetime(df1['CreatedDatetime']).dt.date
    df1['EffectiveDate'] = pd.to_datetime(df1['EffectiveDate']).dt.date
    df1.loc[(df1['ExpiryDate'].isna()), 'ExpiryDate'] = '1/1/1990'
    df1['ExpiryDate'] = pd.to_datetime(df1['ExpiryDate']).dt.date
    df1['LastModifiedDatetime'] = pd.to_datetime(df1['LastModifiedDatetime']).dt.date
    
    df1 = df1.where(pd.notnull(df1), None)
    
    df1.loc[~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['CreatedDatetime'].transform(max) == df1['CreatedDatetime']), 'CreatedDatetime_status'] = 'Previous'
    df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['CreatedDatetime'].transform(max) == df1['CreatedDatetime']), 'CreatedDatetime_status'] = 'Latest'
        
    df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & ~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(min) == df1['ExpiryDate']), 'ExpiryDate_status'] = 'Latest'
    df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & ((df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(min) == df1['ExpiryDate'])), 'ExpiryDate_status'] = 'Current'
    df1.loc[~(df1['CreatedDatetime_status'] == 'Latest') & ((df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(min) == df1['ExpiryDate'])), 'ExpiryDate_status'] = 'Previous CD but Max & Min Exp Date Error'
    df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(max) != df1['ExpiryDate']), 'ExpiryDate_status'] = 'Latest CD but not Max Exp Date Error'
    df1.loc[~(df1['CreatedDatetime_status'] == 'Latest') & ((df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & ~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(min) == df1['ExpiryDate'])), 'ExpiryDate_status'] = 'Previous CD but Max & not Min Exp Date Error'
    df1.loc[~(df1['CreatedDatetime_status'] == 'Latest') & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(max) != df1['ExpiryDate']), 'ExpiryDate_status'] = 'Previous CD and not Max Exp Date Error'
    df1.loc[pd.to_datetime(df1['ExpiryDate']) == '1990-01-01', 'ExpiryDate_status'] += ' & Null Placeholder'
    df1.loc[(pd.to_datetime(df1['ExpiryDate']) != '1990-01-01') & (pd.to_datetime(date.today()) >= df1['ExpiryDate']), 'ExpiryDate_status'] += ' & Expired'
    
    df1.loc[~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max) == df1['EffectiveDate']), 'EffectiveDate_status'] = 'Previous EffectiveDate'
    df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max) == df1['EffectiveDate']), 'EffectiveDate_status'] = 'Current EffectiveDate'
    df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max) == df1['EffectiveDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['LastModifiedDatetime'].transform(max) == df1['LastModifiedDatetime']), 'EffectiveDate_status'] += ' & Latest MD'
    df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max) == df1['EffectiveDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['LastModifiedDatetime'].transform(max) != df1['LastModifiedDatetime']), 'EffectiveDate_status'] += ' & Previous MD'
    
    df1.loc[(df1['EffectiveDate']) < (df1['CreatedDatetime']), 'EffectiveDate_status'] += ' & Eff Date less than CD Anomaly'
    df1.loc[(((df1['EffectiveDate']) >= (df1['ExpiryDate'])) & ~(pd.to_datetime(df1['ExpiryDate']) == '1990-01-01')), 'EffectiveDate_status'] += ' & Eff Date greater than/Equal to Exp Date Error'

    todayv2 = datetime.strptime(str(today), '%Y-%m-%d').strftime('%Y-%m-%d')
    todayv3 = datetime.strptime(str(today), '%Y-%m-%d').strftime('%m-%d')
    last_monthv2 = datetime.strptime(str(last_month), '%Y-%m-%d').strftime('%Y-%m-%d')
    df1['ExpiryDateFirstOfPreviousMonthStr'] = ((df1['ExpiryDate']) - relativedelta(months=1)).astype('str')
    df1['ExpiryDateMaxEffDateMinus1'] = df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max)
    df1['ExpiryDateMaxEffDateMinus1'] = ((df1['ExpiryDateMaxEffDateMinus1']) - relativedelta(days=1)).astype('str')
    df1['Update_Statement'] = 'update resourcepersonrate set'
    df1.loc[df1['ExpiryDate_status'].str.contains('Expired') , 'Update_Statement'] += " expirydate = '1990-01-01'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD & Eff Date greater than/Equal to Exp Date Error'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
    df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '"+df1['ExpiryDateMaxEffDateMinus1']+"'"
    df1.loc[df1['Update_Statement'] == 'update resourcepersonrate set' , 'Update_Statement'] = ''
    df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f", lastmodifieddatetime = '{todayv2}'"
    df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f", comments = comments +' | edit from {todayv3}'"
    df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f" where resourcepersonratesk = '"+df1['Resourcepersonratesk'].astype('str')+"'"
    
    df3_nofilter = df1[['PersonPartyID','LegalEntityPartyID','rateType','EffectiveDate','ExpiryDate','CreatedDatetime','LastModifiedDatetime','RateBasis','CreatedDatetime_status','ExpiryDate_status','EffectiveDate_status','Update_Statement']]
    
    display(df3_nofilter)
    
    return

In [6]:
duplication_extraction()

<ipython-input-4-90a4e11210d9>:38: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df1pre = pd.read_csv(f'DataAndDuplicateReportAutomation/{processlist[0]}')
<ipython-input-4-90a4e11210d9>:68: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df1.loc[(pd.to_datetime(df1['ExpiryDate']) != '1990-01-01') & (pd.to_datetime(date.today()) >= df1['ExpiryDate']), 'ExpiryDate_status'] += ' & Expired'


NameError: name 'df2_nofilter' is not defined

In [10]:
duplication_extraction_withoutcategory()

<ipython-input-9-c498fc45e95e>:38: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df1pre = pd.read_csv(f'DataAndDuplicateReportAutomation/{processlist[0]}')
<ipython-input-9-c498fc45e95e>:66: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df1.loc[(pd.to_datetime(df1['ExpiryDate']) != '1990-01-01') & (pd.to_datetime(date.today()) >= df1['ExpiryDate']), 'ExpiryDate_status'] += ' & Expired'


,PersonPartyID,LegalEntityPartyID,rateType,EffectiveDate,ExpiryDate,CreatedDatetime,LastModifiedDatetime,RateBasis,CreatedDatetime_status,ExpiryDate_status,EffectiveDate_status,Update_Statement
94673,0000FB73-0624-4E47-A87B-72BC7D1DA06F,F30ABFAB-4C61-4F45-B40C-1AB13706FD3C,Cost,2022-10-01,1990-01-01,2022-10-04,2022-10-05,Resource Role,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,
94674,0000FB73-0624-4E47-A87B-72BC7D1DA06F,F30ABFAB-4C61-4F45-B40C-1AB13706FD3C,Scale,2022-10-01,1990-01-01,2022-10-04,2022-10-05,Resource Role,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,
48373,000235DB-D90F-4881-8774-8BB5C9D8EB76,5715AD3E-BF5D-46A2-A831-E68192667608,Scale,2022-09-12,1990-01-01,2022-09-01,2022-09-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
48374,000235DB-D90F-4881-8774-8BB5C9D8EB76,5715AD3E-BF5D-46A2-A831-E68192667608,Cost,2022-09-12,1990-01-01,2022-09-01,2022-09-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
104336,0005E8E3-8346-4353-8467-E99C1824F9C2,14EE24E4-336A-4C9D-8A3F-CC5FF691C6CE,Cost,2022-09-01,1990-01-01,2022-09-01,2022-09-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
...,...,...,...,...,...,...,...,...,...,...,...,...
25651,FFFCB7DF-4463-4344-9F39-530FD1267597,F21F2D2F-FF8B-4792-9271-839A91658589,Scale,2022-07-01,1990-01-01,2022-07-01,2022-07-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
87692,FFFEE8EE-33F8-4B1F-8E59-F89F156F6854,FF6D5AF8-9AE6-47D9-8317-EAA8B0E2E7B9,Scale,2022-07-01,1990-01-01,2022-07-02,2022-07-02,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,
87693,FFFEE8EE-33F8-4B1F-8E59-F89F156F6854,FF6D5AF8-9AE6-47D9-8317-EAA8B0E2E7B9,Cost,2022-08-01,1990-01-01,2022-07-19,2022-07-21,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
3132,FFFEF5FC-F5AF-4BE4-A737-896EB5C175EF,B886A7F0-9C89-48A9-BE0B-EB2337697C8D,Scale,2022-04-22,1990-01-01,2022-04-21,2022-04-21,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,


In [122]:
today = date.today()
first = today.replace(day=1)
last_month = first - timedelta(days=1)
tenth_of_current_month = today.replace(day=10)
first_of_previous_month = last_month.replace(day=1)
next_month = today.replace(day=28) + timedelta(days=4)
next_month = next_month - timedelta(days=next_month.day)

todayv2 = datetime.strptime(str(today), '%Y-%m-%d').strftime('%Y-%m-%d')
todayv3 = datetime.strptime(str(today), '%Y-%m-%d').strftime('%m-%d')
last_monthv2 = datetime.strptime(str(last_month), '%Y-%m-%d').strftime('%Y-%m-%d')
df1['ExpiryDateFirstOfPreviousMonthStr'] = ((df1['ExpiryDate']) - relativedelta(months=1)).astype('str')
df1['ExpiryDateMaxEffDateMinus1'] = df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max)
df1['ExpiryDateMaxEffDateMinus1'] = ((df1['ExpiryDateMaxEffDateMinus1']) - relativedelta(days=1)).astype('str')
df1['Update_Statement'] = 'update resourcepersonrate set'
df1.loc[df1['ExpiryDate_status'].str.contains('Expired') , 'Update_Statement'] += " expirydate = '1990-01-01'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD & Eff Date greater than/Equal to Exp Date Error'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '"+df1['ExpiryDateMaxEffDateMinus1']+"'"
df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1['ExpiryDate_status'] == 'Current & Null Placeholder') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '"+df1['ExpiryDateMaxEffDateMinus1']+"'"
df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1['ExpiryDate_status'] == 'Current & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '"+df1['ExpiryDateMaxEffDateMinus1']+"'"
df1.loc[df1['Update_Statement'] == 'update resourcepersonrate set' , 'Update_Statement'] = ''
df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f", lastmodifieddatetime = '{todayv2}'"
df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f", comments = comments +' | edit from {todayv3} correction automation'"
df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f" where resourcepersonratesk = '"+df1['Resourcepersonratesk'].astype('str')+"'"

df2 = df1[['PersonPartyID','LegalEntityPartyID','rateType','EffectiveDate','ExpiryDate','CreatedDatetime','LastModifiedDatetime','RateBasis','CreatedDatetime_status','ExpiryDate_status','EffectiveDate_status','Update_Statement']]

#,'PartyID_RateRule','RateRule_Eff_Date','RateRule_Exp_Date','Rate_Basis_Count','ExpiryDate_update_S1','ExpiryDate_update_S2','ExpiryDate_update_equaldates',

#UpdExp = "update resourcepersonrate set expirydate = '2022-09-30', lastmodifieddatetime = No, comments = comments +':11/8' where resourcepersonratesk = 20550402"
#UpdLMD = "update resourcepersonrate set lastmodifieddatetime = getdate(), comments = '11/8 LMD' where resourcepersonratesk = 27151709"

df2e = df1.loc[df1['Update_Statement'].notna()]
display(df2e)
print(len(df2e))

#display(df2)

#df1.to_csv('DataAndDuplicateReportAutomation/Analysis/updatesfrom0.97.csv', index = False)

#Below are Not In Error

#Previous / Previous CD and not Max Exp Date Error & Null Placeholder / Current EffectiveDate & Latest MD
#Latest / Latest CD but not Max Exp Date Error & Null Placeholder / Current EffectiveDate & Latest MD
#Latest / Latest / Previous EffectiveDate
#Previous / Previous CD but Max & not Min Exp Date Error / Previous EffectiveDate
#Previous / Previous CD but Max & not Min Exp Date Error / Current EffectiveDate & Previous MD



#Latest	Current & Null Placeholder	Current EffectiveDate & Previous MD & Eff Date less than CD Anomaly - ANOMALY FROM EDIT - Awaiting response from joseph
#Latest	Current & Null Placeholder	Previous EffectiveDate - ANOMALY FROM EDIT - Awaiting response from joseph

#Done
#Latest	Current & Null Placeholder	Previous EffectiveDate & Eff Date less than CD Anomaly - ANOMALY FROM EDIT


#UpdExp = "update resourcepersonrate set expirydate = '2022-09-30', lastmodifieddatetime = No, comments = comments +':11/8' where resourcepersonratesk = 20550402"
#UpdLMD = "update resourcepersonrate set lastmodifieddatetime = getdate(), comments = '11/8 LMD' where resourcepersonratesk = 27151709"

#Latest	Current & Null Placeholder	Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly - Good
#Latest	Current & Null Placeholder	Current EffectiveDate & Latest MD - Good
#Latest	Latest CD but not Max Exp Date Error & Null Placeholder	Current EffectiveDate & Latest MD - Good
#Latest	Latest CD but not Max Exp Date Error & Null Placeholder	Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly - Good
#Latest	Latest CD but not Max Exp Date Error	Current EffectiveDate & Previous MD - Good 
#Latest	Latest CD but not Max Exp Date Error & Null Placeholder	Current EffectiveDate & Previous MD - Good
#Latest	Latest CD but not Max Exp Date Error	Current EffectiveDate & Previous MD & Eff Date less than CD Anomaly - Good 
#Latest	Latest CD but not Max Exp Date Error & Null Placeholder	Current EffectiveDate & Previous MD & Eff Date less than CD Anomaly - Good
#Latest	Latest CD but not Max Exp Date Error & Null Placeholder	Previous EffectiveDate - Good


#Current EffectiveDate & Previous MD & Eff Date less than CD Anomaly - 14F6C365-4961-4896-884D-3260D2A4E281 - 47719958-3432-42CE-97B8-A7C410098B0F
#Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly - A7567D04-3241-45F7-A410-6A8B56F68A8E - C976D789-AA2F-4EF6-BD8C-E5A7C3E1C468
#A7567D04-3241-45F7-A410-6A8B56F68A8E
#display(df1)
df3 = df2.copy()
display(df2)

,PwCNetworkUID,PwCNetworkNodeName,PersonPartyID,LegalEntityPartyID,EmailAddress,LocalCostCenterCode,ResourceRole,GlobalGrade,InternalpersonCategoryname,workerType,rateType,DupRate,Rate,EffectiveDate,ExpiryDate,Resourcepersonratesk,RateRulesk,CreatedDatetime,LastModifiedDatetime,Raterulesk,ActiveRecords_attributeapping,ActiveRecords_Rate,RateBasis,Comments,PartyID_RateRule,RateRule_Eff_Date,RateRule_Exp_Date,CreatedDatetime_status,ExpiryDate_status,EffectiveDate_status,ExpiryDateFirstOfPreviousMonthStr,ExpiryDateMaxEffDateMinus1,Update_Statement
94673,1974667,PwC Netherlands,0000FB73-0624-4E47-A87B-72BC7D1DA06F,F30ABFAB-4C61-4F45-B40C-1AB13706FD3C,zezheng.gan@pwc.com,NL12000412,Senior Associate 2,Senior Associate 2,Client Service,PwC Worker,Cost,No,106,2022-10-01,1990-01-01,23427026,13498079,2022-10-04,2022-10-05,13498079,1.0,1,Resource Role,October 2022 promotions cleanup,None,None,NaN,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,1989-12-01,2022-09-30,
94674,1974667,PwC Netherlands,0000FB73-0624-4E47-A87B-72BC7D1DA06F,F30ABFAB-4C61-4F45-B40C-1AB13706FD3C,zezheng.gan@pwc.com,NL12000412,Senior Associate 2,Senior Associate 2,Client Service,PwC Worker,Scale,No,450,2022-10-01,1990-01-01,23427226,13498087,2022-10-04,2022-10-05,13498087,1.0,1,Resource Role,October 2022 promotions cleanup,None,None,NaN,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,1989-12-01,2022-09-30,
48373,1974664,PwC Austria,000235DB-D90F-4881-8774-8BB5C9D8EB76,5715AD3E-BF5D-46A2-A831-E68192667608,simon.l.letko@pwc.com,AT18000019,Specialist,Specialist 5,Client Service,PwC Worker,Scale,No,128,2022-09-12,1990-01-01,19259351,1124088,2022-09-01,2022-09-01,1124088,1.0,1,Job Level,None,None,None,NaN,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,1989-12-01,2022-09-11,
48374,1974664,PwC Austria,000235DB-D90F-4881-8774-8BB5C9D8EB76,5715AD3E-BF5D-46A2-A831-E68192667608,simon.l.letko@pwc.com,AT18000019,Specialist,Specialist 5,Client Service,PwC Worker,Cost,No,64,2022-09-12,1990-01-01,19259323,1125707,2022-09-01,2022-09-01,1125707,1.0,1,Job Level,None,None,None,NaN,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,1989-12-01,2022-09-11,
104336,1974718,PwC Kenya,0005E8E3-8346-4353-8467-E99C1824F9C2,14EE24E4-336A-4C9D-8A3F-CC5FF691C6CE,naeku.edith@pwc.com,KE0183,Associate,Associate,Client Service,PwC Worker,Cost,No,3744,2022-09-01,1990-01-01,19320212,18237286,2022-09-01,2022-09-01,18237286,NaN,1,Job Level,None,None,None,NaN,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,1989-12-01,2022-08-31,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25651,1974640,PwC Thailand,FFFCB7DF-4463-4344-9F39-530FD1267597,F21F2D2F-FF8B-4792-9271-839A91658589,napapan.thaveeptavonvong@pwc.com,BKK100,Associate,Associate,Client Service,PwC Worker,Scale,No,1500,2022-07-01,1990-01-01,16438969,16402640,2022-07-01,2022-07-01,16402640,1.0,1,Job Level,None,None,None,NaN,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,1989-12-01,2022-06-30,
87692,1974666,PwC Germany,FFFEE8EE-33F8-4B1F-8E59-F89F156F6854,FF6D5AF8-9AE6-47D9-8317-EAA8B0E2E7B9,marc.mertens@pwc.com,DE01000269,Intern/Trainee,Intern/Trainee,Client Service,PwC Worker,Scale,No,68,2022-07-01,1990-01-01,16441196,16196368,2022-07-02,2022-07-02,16196368,1.0,1,Job Level,None,None,None,NaN,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,1989-12-01,2022-06-30,
87693,1974666,PwC Germany,FFFEE8EE-33F8-4B1F-8E59-F89F156F6854,FF6D5AF8-9AE6-47D9-8317-EAA8B0E2E7B9,marc.mertens@pwc.com,DE01000269,Intern/Trainee,Intern/Trainee,Client Service,PwC Worker,Cost,No,17,2022-08-01,1990-01-01,17742854,17311219,2022-07-19,2022-07-21,17311219,1.0,1,Job Level,None,None,None,NaN,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,1989-12-01,2022-07-31,
3132,1974576,PwC New Zealand,FFFE

122269


,PersonPartyID,LegalEntityPartyID,rateType,EffectiveDate,ExpiryDate,CreatedDatetime,LastModifiedDatetime,RateBasis,CreatedDatetime_status,ExpiryDate_status,EffectiveDate_status,Update_Statement
94673,0000FB73-0624-4E47-A87B-72BC7D1DA06F,F30ABFAB-4C61-4F45-B40C-1AB13706FD3C,Cost,2022-10-01,1990-01-01,2022-10-04,2022-10-05,Resource Role,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,
94674,0000FB73-0624-4E47-A87B-72BC7D1DA06F,F30ABFAB-4C61-4F45-B40C-1AB13706FD3C,Scale,2022-10-01,1990-01-01,2022-10-04,2022-10-05,Resource Role,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,
48373,000235DB-D90F-4881-8774-8BB5C9D8EB76,5715AD3E-BF5D-46A2-A831-E68192667608,Scale,2022-09-12,1990-01-01,2022-09-01,2022-09-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
48374,000235DB-D90F-4881-8774-8BB5C9D8EB76,5715AD3E-BF5D-46A2-A831-E68192667608,Cost,2022-09-12,1990-01-01,2022-09-01,2022-09-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
104336,0005E8E3-8346-4353-8467-E99C1824F9C2,14EE24E4-336A-4C9D-8A3F-CC5FF691C6CE,Cost,2022-09-01,1990-01-01,2022-09-01,2022-09-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
...,...,...,...,...,...,...,...,...,...,...,...,...
25651,FFFCB7DF-4463-4344-9F39-530FD1267597,F21F2D2F-FF8B-4792-9271-839A91658589,Scale,2022-07-01,1990-01-01,2022-07-01,2022-07-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
87692,FFFEE8EE-33F8-4B1F-8E59-F89F156F6854,FF6D5AF8-9AE6-47D9-8317-EAA8B0E2E7B9,Scale,2022-07-01,1990-01-01,2022-07-02,2022-07-02,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,
87693,FFFEE8EE-33F8-4B1F-8E59-F89F156F6854,FF6D5AF8-9AE6-47D9-8317-EAA8B0E2E7B9,Cost,2022-08-01,1990-01-01,2022-07-19,2022-07-21,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,
3132,FFFEF5FC-F5AF-4BE4-A737-896EB5C175EF,B886A7F0-9C89-48A9-BE0B-EB2337697C8D,Scale,2022-04-22,1990-01-01,2022-04-21,2022-04-21,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,


In [106]:
pd.set_option('display.width', 3000)
df2e = df2.loc[df1['Update_Statement'] != '']
display(df2e)
print(len(df2e))

,PersonPartyID,LegalEntityPartyID,rateType,EffectiveDate,ExpiryDate,CreatedDatetime,LastModifiedDatetime,RateBasis,CreatedDatetime_status,ExpiryDate_status,EffectiveDate_status,Update_Statement
4762,003928EF-A251-4DF9-A672-E8DF0B49D08C,B886A7F0-9C89-48A9-BE0B-EB2337697C8D,Scale,2022-07-01,2022-11-28,2022-07-03,2022-11-16,Job Level,Latest,Current & Expired,Current EffectiveDate & Latest MD & Eff Date l...,update resourcepersonrate set expirydate = '19...
4763,003928EF-A251-4DF9-A672-E8DF0B49D08C,B886A7F0-9C89-48A9-BE0B-EB2337697C8D,Cost,2022-10-01,2022-11-28,2022-09-30,2022-11-16,Job Level,Latest,Current & Expired,Current EffectiveDate & Latest MD,update resourcepersonrate set expirydate = '19...
41629,00C66D0C-6004-40E1-A187-6858C21EE44A,EA5D8532-B488-4270-8E77-ADC40703771F,Cost,2022-09-01,2022-12-31,2021-12-23,2022-10-25,Job Level,Previous,Previous CD but Max & Min Exp Date Error,Previous EffectiveDate,update resourcepersonrate set expirydate = '20...
41630,00C66D0C-6004-40E1-A187-6858C21EE44A,EA5D8532-B488-4270-8E77-ADC40703771F,Scale,2022-09-01,2022-12-31,2021-12-23,2022-10-25,Job Level,Previous,Previous CD but Max & Min Exp Date Error,Previous EffectiveDate,update resourcepersonrate set expirydate = '20...
99694,0141FE39-2A59-4A39-8F85-D1F6BBDDD52A,F30ABFAB-4C61-4F45-B40C-1AB13706FD3C,Cost,2022-07-01,2022-11-30,2022-05-24,2022-10-13,Resource Role,Previous,Previous CD but Max & not Min Exp Date Error &...,Previous EffectiveDate,update resourcepersonrate set expirydate = '19...
...,...,...,...,...,...,...,...,...,...,...,...,...
113686,FF91742B-36DB-41E3-866A-4247145AE47C,12D8572E-11F3-49B4-825B-460FFF699F79,Scale,2022-09-01,2022-11-30,2022-08-30,2022-11-14,Job Level,Latest,Current & Expired,Current EffectiveDate & Latest MD,update resourcepersonrate set expirydate = '19...
92670,FF93EA61-2C51-44A7-B26F-4B6F0CFE64AF,3E49BA1C-F736-4C18-A391-8CE71B8F20F3,Scale,2022-10-01,2022-11-30,2022-09-09,2022-11-07,Resource Role,Previous,Previous CD but Max & not Min Exp Date Error &...,Previous EffectiveDate,update resourcepersonrate set expirydate = '19...
92671,FF93EA61-2C51-44A7-B26F-4B6F0CFE64AF,3E49BA1C-F736-4C18-A391-8CE71B8F20F3,Cost,2022-10-01,2022-11-30,2022-09-09,2022-11-07,Resource Role,Previous,Previous CD but Max & not Min Exp Date Error &...,Previous EffectiveDate,update resourcepersonrate set expirydate = '19...
46565,FFD4DCAE-D3BB-4B2D-AED8-F9266DBE4CA3,EA5D8532-B488-4270-8E77-ADC40703771F,Scale,2022-10-24,2022-11-30,2022-10-21,2022-11-15,Resource Role,Previous,Previous CD but Max & not Min Exp Date Error &...,Previous EffectiveDate,update resourcepersonrate set expirydate = '19...


1538


In [112]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

dfsearch = df1.loc[df1['PersonPartyID'].isin(['3DC464A8-A2B3-47FA-844D-3326488CD377'])]
display(dfsearch)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

dfsearch4 = df1.loc[df1['PersonPartyID'].isin(['3DC464A8-A2B3-47FA-844D-3326488CD377',
'570F6D64-2A9E-48C2-97FD-CDA56E4B3D86',
'791C502B-232A-47DB-B21D-49D35D87EED0',
'792A0D97-4B59-4E0C-B75A-71BB4BE5509D',
'81941923-CCEB-4F18-84A6-58B5D3D2F758',
'8719761A-B2E8-4719-BC01-62655408D325',
'882BA7D9-8815-4BD8-BF85-AE9ABC24362A',
'8951DA78-0D11-47AF-BBAB-E6BEAD387012',
'9AE4081D-C690-4772-9666-982D8C6D965A',
'9B3CB664-A1FB-44B2-9317-128232636918',
'AE539C0F-44E0-4AEC-965A-A4B5FBA040EA',
'BD00B570-3B1A-4E6E-95D8-49D2C7F32860',
'C632C947-8346-4DEA-9CA4-B69BE91335A0',
'C6A4367F-ABA2-481A-92CF-167A19A4D8EA',
'E7D6CA14-3B0A-412E-956E-03003508AB2A',
'E802546B-0D61-4DAC-A5A9-9ED127721001',
'E85EFB61-7E47-40F1-B307-F3331692F846',
'E96BA331-79CE-4BAF-91A1-DE1F4316B4EB',
'EDAD9A5B-B779-4C8D-8F04-5A130336A5B0',
'EECE6B5E-3B64-4874-9643-B4E55AF37A86'])]
#display(dfsearch4)

#dfsearch4.to_csv('DataAndDuplicateReportAutomation/Analysis/Anomaly.csv', index = False)

dfsearch5 = df1.loc[df1['PersonPartyID'].isin(['3DC464A8-A2B3-47FA-844D-3326488CD377',
'570F6D64-2A9E-48C2-97FD-CDA56E4B3D86',
'791C502B-232A-47DB-B21D-49D35D87EED0',
'792A0D97-4B59-4E0C-B75A-71BB4BE5509D',
'81941923-CCEB-4F18-84A6-58B5D3D2F758',
'8719761A-B2E8-4719-BC01-62655408D325',
'882BA7D9-8815-4BD8-BF85-AE9ABC24362A',
'8951DA78-0D11-47AF-BBAB-E6BEAD387012',
'9AE4081D-C690-4772-9666-982D8C6D965A',
'9B3CB664-A1FB-44B2-9317-128232636918',
'AE539C0F-44E0-4AEC-965A-A4B5FBA040EA',
'BD00B570-3B1A-4E6E-95D8-49D2C7F32860',
'C632C947-8346-4DEA-9CA4-B69BE91335A0',
'C6A4367F-ABA2-481A-92CF-167A19A4D8EA',
'E7D6CA14-3B0A-412E-956E-03003508AB2A',
'E802546B-0D61-4DAC-A5A9-9ED127721001',
'E85EFB61-7E47-40F1-B307-F3331692F846',
'E96BA331-79CE-4BAF-91A1-DE1F4316B4EB',
'EDAD9A5B-B779-4C8D-8F04-5A130336A5B0',
'EECE6B5E-3B64-4874-9643-B4E55AF37A86']) & df1['InternalpersonCategoryname'].str.contains('Practice Support')]
#display(dfsearch5)

#dfsearch5.to_csv('DataAndDuplicateReportAutomation/Analysis/AnomalyCause.csv', index = False)

,PwCNetworkUID,PwCNetworkNodeName,PersonPartyID,LegalEntityPartyID,EmailAddress,LocalCostCenterCode,ResourceRole,GlobalGrade,InternalpersonCategoryname,workerType,rateType,DupRate,Rate,EffectiveDate,ExpiryDate,Resourcepersonratesk,RateRulesk,CreatedDatetime,LastModifiedDatetime,Raterulesk,ActiveRecords_attributeapping,ActiveRecords_Rate,RateBasis,Comments,PartyID_RateRule,RateRule_Eff_Date,RateRule_Exp_Date,CreatedDatetime_status,ExpiryDate_status,EffectiveDate_status,ExpiryDateFirstOfPreviousMonthStr,ExpiryDateMaxEffDateMinus1,Update_Statement
25154,1974640,PwC Thailand,3DC464A8-A2B3-47FA-844D-3326488CD377,F21F2D2F-FF8B-4792-9271-839A91658589,pajaree.t.thienmontree@pwc.com,BKK160,Senior Associate,Senior Associate,Client Service,PwC Worker,Cost,No,730,2022-07-01,1990-01-01,16405160,16404109,2022-07-01,2022-11-21,16404109,1.0,1,Job Level,October 17 2022 promotions cleanup: October 19...,None,None,NaN,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,1989-12-01,2022-06-30,
25155,1974640,PwC Thailand,3DC464A8-A2B3-47FA-844D-3326488CD377,F21F2D2F-FF8B-4792-9271-839A91658589,pajaree.t.thienmontree@pwc.com,BKK160,Senior Associate,Senior Associate,Client Service,PwC Worker,Scale,Yes,2500,2022-07-01,1990-01-01,16405712,16401409,2022-07-01,2022-11-21,16401409,1.0,2,Job Level,October 17 2022 promotions cleanup: October 19...,None,None,NaN,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,1989-12-01,2022-06-30,
25156,1974640,PwC Thailand,3DC464A8-A2B3-47FA-844D-3326488CD377,F21F2D2F-FF8B-4792-9271-839A91658589,pajaree.t.thienmontree@pwc.com,BKK160,Senior Associate,Senior Associate,Practice Support,PwC Worker,Scale,No,2500,2022-07-01,1990-01-01,16888587,16403671,2022-07-14,2022-07-14,16403671,1.0,2,Job Level,None,None,None,NaN,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,1989-12-01,2022-06-30,


In [11]:
df4 = pd.merge(df2_filter, df3_nofilter, how = "outer", indicator=True)
display(df4)
df4.to_csv('DataAndDuplicateReportAutomation/Analysis/df4.csv', index = False)

,PersonPartyID,LegalEntityPartyID,rateType,EffectiveDate,ExpiryDate,CreatedDatetime,LastModifiedDatetime,RateBasis,CreatedDatetime_status,ExpiryDate_status,EffectiveDate_status,Update_Statement,_merge
0,0000FB73-0624-4E47-A87B-72BC7D1DA06F,F30ABFAB-4C61-4F45-B40C-1AB13706FD3C,Cost,2022-10-01,1990-01-01,2022-10-04,2022-10-05,Resource Role,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,,both
1,0000FB73-0624-4E47-A87B-72BC7D1DA06F,F30ABFAB-4C61-4F45-B40C-1AB13706FD3C,Scale,2022-10-01,1990-01-01,2022-10-04,2022-10-05,Resource Role,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD & Eff Date l...,,both
2,000235DB-D90F-4881-8774-8BB5C9D8EB76,5715AD3E-BF5D-46A2-A831-E68192667608,Scale,2022-09-12,1990-01-01,2022-09-01,2022-09-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,,both
3,000235DB-D90F-4881-8774-8BB5C9D8EB76,5715AD3E-BF5D-46A2-A831-E68192667608,Cost,2022-09-12,1990-01-01,2022-09-01,2022-09-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,,both
4,0005E8E3-8346-4353-8467-E99C1824F9C2,14EE24E4-336A-4C9D-8A3F-CC5FF691C6CE,Cost,2022-09-01,1990-01-01,2022-09-01,2022-09-01,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Latest MD,,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122352,E96BA331-79CE-4BAF-91A1-DE1F4316B4EB,F21F2D2F-FF8B-4792-9271-839A91658589,Scale,2022-07-01,1990-01-01,2022-07-15,2022-07-15,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Previous MD & Eff Date...,,right_only
122353,EDAD9A5B-B779-4C8D-8F04-5A130336A5B0,F21F2D2F-FF8B-4792-9271-839A91658589,Scale,2022-07-01,1990-01-01,2022-07-01,2022-11-21,Job Level,Previous,Previous CD but Max & Min Exp Date Error & Nul...,Current EffectiveDate & Latest MD,update resourcepersonrate set expirydate = '20...,right_only
122354,EDAD9A5B-B779-4C8D-8F04-5A130336A5B0,F21F2D2F-FF8B-4792-9271-839A91658589,Scale,2022-07-01,1990-01-01,2022-07-04,2022-07-04,Job Level,Latest,Current & Null Placeholder,Current EffectiveDate & Previous MD & Eff Date...,,right_only
122355,EECE6B5E-3B64-4874-9643-B4E55AF37A86,B886A7F0-9C89-48A9-BE0B-EB2337697C8D,Scale,2022-07-01,1990-01-01,2022-07-05,2022-11-21,Job Level,Previous,Previous CD but Max & Min Exp Date Error & Nul...,Current EffectiveDate & Latest MD & Eff Date l...,update resourcepersonrate set expirydate = '20...,right_only


In [113]:
from dateutil.relativedelta import relativedelta
import numpy as np
import time as ti
import datetime
from datetime import datetime, date, timedelta

df1.to_csv('DataAndDuplicateReportAutomation/Analysis/Analysis0.98.csv', index = False)

In [6]:
process_CSV()

Analysis
Archive
DataAndDuplicateReport_2022-11-21_19-01-29.csv

DataAndDuplicateReport_2022-11-21_19-01-29.csv
Temp


['DataAndDuplicateReport_2022-11-21_19-01-29.csv']

In [13]:
df1.to_csv('DataAndDuplicateReportAutomation/Analysis/Analysis0.5reducedfields.csv', index = False)